# Generate Product Description - Text Generation
<br>

Before starting, please make sure this notebook is using **conda_python3** kernel from the top right!

To run this notebook, go to **Cell -> Run All**. Read the comments in the notebook and inspect the output of each cell.

<b> Please read the following instructions carefully! </b><br>

Following instructions are common to ALL the notebooks you will run in this workshop.

* You will use the Jupyter notebooks for understanding the code you will be implementing in the Cloud9 IDE. 
* These notebooks are not a part of the retail web application deployment process.
* To save time and avoid any issues, we highly recommend you to run ALL cells and inspect output of each cell rather than running the cells individually.
* Limit your experimentations within the notebook since we have a dedicated **Explore** section. 
* Within the notebooks, we have provided brief explanation of all the core concepts and terminologies you need for this workshop. 
    * If you are already aware of these concepts, feel free to skip reading the explanations.
* We have provided many hyperlinks or URLs throughout the notebooks for the purpose of further learning. 
    * In the interest of time, we request you to open and read these URLs **after** today's workshop i.e., as **take home** materials, as much as possible. 
    * You can always find this notebook (and all other notebooks) in this [Github repository](https://github.com/aws-samples/retails-generative-ai-workshop/tree/main/notebooks) which contains these URLs. Feel free to bookmark the Github repo for later reference.

### Introduction

In this notebook, you will generate product description using Text-to-Text LLMs (Large Language Models) from Amazon Bedrock. Let's take a quick look into the two important concepts we will use in all of our notebooks. 

[LangChain](https://python.langchain.com/docs/get_started/introduction) is an opensource framework used for developing applications powered by LLMs (Large Language Models). LangChain provides many libraries, APIs and templates to make it easy for us to develop applications powered by LLMs. LangChain [integrates with Bedrock](https://python.langchain.com/docs/integrations/llms/bedrock), and can be used to invoke the Bedrock APIs. **Note:** We will be using LangChain for 4 out of the 6 features in this workshop. 

A prompt is natural language text that describes the task that an LLM should perform. For example, you may prompt the LLM: *Describe microbiomes* or *What is the abstract of this text?*. You can use prompts to do various tasks with LLM such as Text Generation, Text Summarization, Sentiment Analysis, Classification etc. [Here](https://www.promptingguide.ai/introduction/examples) are some additional prompting examples for later reference. 

To generate product description, we will create a prompt with a few input variables, and invoke Bedrock LLM using LangChain. 

![Text Generation](../images/text-generation.png)

### Install required dependencies

The pip install command may take 1-2 minutes to run.

**Important:** You may notice an **ERROR** and a warning that "you may need to restart the kernel" from the following cell. **Ignore** and proceed with the next cells. 

In [ ]:
%pip install --quiet --no-build-isolation --force-reinstall \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" \
    "langchain==0.0.309"

<h3> Import required packages </h3>

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3><br>

We will use this client to invoke the Bedrock API and call the LLM. 

In [ ]:
 boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

### Set inference parameters for LLM

Every LLM has its own set of [inference parameters](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html). Setting infence parameters values are optional (as they have defaults). 

Following are the inference parameters for [Claude Anthropic LLM](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html). Note that we will describe more about these parameters in the **Explore** section of the workshop.

In [ ]:
inference_modifier = {}
inference_modifier['max_tokens_to_sample'] = 200
inference_modifier['temperature'] = 0.5
inference_modifier['top_k'] = 250
inference_modifier['top_p'] = 1
inference_modifier['stop_sequences'] = ["\n\nHuman"]

<h3> Initialize the LLM </h3>
<p> Using Langchain, initialize the ClaudeInstant LLM for text generation. </p>

In [ ]:
# Passing Anthropic Claude as model ID

textgen_llm = Bedrock(
            model_id="anthropic.claude-instant-v1",
            client=boto3_bedrock,
            model_kwargs=inference_modifier,
        )

### Prompt template

[Prompt template](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) offers an easy way for you to work with the prompts. In this case, we use a prompt template to pass in our input variables.  

Let's create a prompt template with the input variables: product name, brand, category, colors available, details, and desired length of the output. Read the template carefully to understand the prompt. </p> 

In [ ]:
prompt_template = PromptTemplate(
            input_variables=["brand", "colors", "category", "length", "name","details"], 
            template="""
            
                Human: Create a catchy product description for a {category} from the brand {brand}. 
                Product name is {name}. 
                The number of words should be less than {length}. 

                Following are the product details:  

                <product_details>
                {details}
                </product_details>

                Briefly mention about all the available colors of the product.

                Example: Available colors are Blue, Purple and Orange. 

                If the <available_colors> is empty, don't mention anything about the color of the product.

                <available_colors>
                {colors}
                </available_colors>

                Assistant:
                
                """
    
            )

### Provide sample details about a product

Following are the input variables that will be passed to our prompt template. This prompt will be passed to the Bedrock LLM to generate product description.

In [ ]:
product_brand = "Legendaire"
product_category = "Shirt"
product_name = "Legendaire Shirt"

product_details = """
        - collared white shirt 
        - 80% cotton 20% polyester
        - semi-casual
        - great for office or golfing
        - comfortable breathable material
        - flex fit
    """

product_colors = ["White", "Black", "Blue"]

# Length of the desired product description (generated from LLM)
max_length = 100

Let's pass the above variables as inputs to the prompt template.

In [ ]:
prompt = prompt_template.format(brand=product_brand, 
                                colors=product_colors,
                                category=product_category,
                                length=max_length,
                                name=product_name,
                                details=product_details)

<h3> Call Bedrock with the constructed prompt </h3>

In [ ]:
response = textgen_llm(prompt)
print_ww(response)

<h4> Get the second paragraph i.e, only the product description </h4>

In [ ]:
generated_response = response[response.index('\n')+1:]
print_ww(generated_response)

#### We can also try to generate the product descripton using another Text-to-Text LLM from Bedrock. Let's use Amazon Titan LLM

In [ ]:
# Initialize Amazon Titan LLM from Bedrock
textgen_llm = Bedrock(
                model_id="amazon.titan-text-express-v1",
                client=boto3_bedrock)

# Call the LLM with same prompt
response = textgen_llm(prompt)
print_ww(response)

### You've successfully generated description for a product with Anthropic Claude LLM!

Please stop the notebook kernel before proceeding with **Kernel -> Interrupt**.

#### Now, let's integrate this feature into our retail web application. Please go back to Workshop Studio and follow the instructions to build this feature using your Cloud9 IDE.